In [1]:
! pip install lxml bs4 requests

In [2]:
import requests
import json
import time
from bs4 import BeautifulSoup
from datetime import datetime

# Lấy thông tin từ portal.ptit.edu.vn

In [3]:
global info_s_file
global anal_info_s
global contens_info_s

info_s_file = 'request_notifications.json'
anal_info_s = 'anal_notifications.json'
contens_info_s = 'contens_notifications.json'

max_page = 26 # Max is 116 with news ang 26 with notifications
# base_url = 'https://portal.ptit.edu.vn/category/tin-tuc/'
base_url = 'https://portal.ptit.edu.vn/category/thong-bao/'

In [4]:
all_news = []

def get_info_s_from_portal(url):
    global all_news
    response = requests.get(url)
    time.sleep(5)

    soup = BeautifulSoup(response.text, from_encoding='utf-8', features='lxml')
    divs = soup.find('div', class_='posts_group lm_wrapper classic col-3')
    info_s = divs.find_all('div')
    
    for item in info_s:
        info_item = {}
        # Tìm tiêu đề từ class "entry-title"
        title_tag = item.find('h2', class_='entry-title')
        info_item['Title'] = title_tag.get_text().strip() if title_tag else ''
        info_item['Link'] = title_tag.find('a')['href'] if title_tag and title_tag.find('a') else ''

        # Tìm nội dung từ class "post-excerpt"
        excerpt_tag = item.find('div', class_='post-excerpt')
        info_item['Content'] = excerpt_tag.get_text().strip() if excerpt_tag else ''

        # Tìm ngày đăng từ class "date_label"
        date_tag = item.find('div', class_='date_label')
        info_item['Date'] = date_tag.get_text().strip() if date_tag else ''

        all_news.append(info_item)

def save_info_s_to_json():
    with open(info_s_file, 'w', encoding='utf-8-sig') as file:
        json.dump(all_news, file, ensure_ascii=False, indent=4)

for i in range(1, max_page + 1):
    get_info_s_from_portal(base_url + 'page/' + str(i))

save_info_s_to_json()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [5]:
# Xử lý dữ liệu JSON
def non_empty_fields(item):
    # Đếm số lượng trường không rỗng trong một item
    non_empty_count = sum(1 for value in item.values() if value)
    return non_empty_count, item

def convert_date(date_str):
    # Chuyển đổi ngày từ chuỗi sang đối tượng datetime, hoặc trả về None nếu không hợp lệ
    try:
        return datetime.strptime(date_str, '%d/%m/%Y')
    except (ValueError, TypeError):
        return None

def process_json_data(data):
    # Tạo list mới chỉ chứa items không hoàn toàn rỗng và thêm 'info_fullness' và 'Date' chuyển đổi
    not_empty_data = [item for item in data if any(item.values())]
    for item in not_empty_data:
        item['info_fullness'] = non_empty_fields(item)[0]
        item['Date'] = convert_date(item['Date']) if item['Date'] else None

    # Sắp xếp list dựa trên 'info_fullness', sau đó theo 'Date' nếu có
    not_empty_sorted = sorted(not_empty_data, key=lambda x: (-x['info_fullness'], x['Date'] if x['Date'] else datetime.min))

    # Loại bỏ những items trùng lặp, giữ lại items với 'info_fullness' cao nhất
    seen = set()
    unique_data = []
    for item in not_empty_sorted:
        # Tạo tuple của 'Title', 'Link' để kiểm tra trùng lặp
        title_link = (item.get('Title'), item.get('Link'))
        if title_link not in seen:
            seen.add(title_link)
            unique_data.append(item)

    # Cuối cùng, sắp xếp kết quả dựa theo 'Date' giảm dần
    final_data = sorted(unique_data, key=lambda x: x['Date'] if x['Date'] else datetime.min, reverse=True)
    
    # Loại bỏ field 'info_fullness' và item nào không có 'Date' hợp lệ
    final_cleaned_data = [item for item in final_data if item['Date']]
    # Chuyển đổi 'Date' thành chuỗi
    for item in final_cleaned_data:
        if item['Date']:
            item['Date'] = item['Date'].strftime('%d/%m/%Y')

    for item in final_cleaned_data:
        del item['info_fullness']
    
    return final_cleaned_data

# Chuẩn bị dữ liệu từ file JSON
with open(info_s_file, 'r', encoding='utf-8-sig') as file:
    data = json.load(file)

# Xử lý dữ liệu
cleaned_data = process_json_data(data)

# Ghi dữ liệu đã được làm sạch vào file JSON mới
with open(anal_info_s, 'w', encoding='utf-8-sig') as file:
    json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

In [ ]:
def get_contents_from_page(url):
    response = requests.get(url)
    time.sleep(5)

    soup = BeautifulSoup(response.text, from_encoding='utf-8', features='lxml')

    div_contents = soup.find('div', class_='the_content_wrapper')
    contents = div_contents.find_all('p')
    
    contens_ = ''
    for content in contents:
         contens_ = ' '.join([contens_, content.get_text().strip()])
    return contens_

# Đọc dữ liệu từ file JSON
with open(anal_info_s, 'r', encoding='utf-8-sig') as file:
    data = json.load(file)

# Lấy nội dung từ mỗi liên kết và thêm vào mỗi item
for item in data:
    contens_info_s_link = item['Link']
    new_content = get_contents_from_page(contens_info_s_link)
    item['Content'] = new_content

# Ghi dữ liệu làm sạch kèm nội dung vào file JSON mới
with open(contens_info_s, 'w', encoding='utf-8-sig') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)